#### beta 12/01/2022 
#### v0 15/02/2022
#### SCIENCE TEAM
#### THE NATURE CONSERVANCY




### JUPYTER NOTEBOOK TO AUTOMATIZE AMAZON SELECTION/PRIORITIZATION AREAS

##### TNC SCIENCE TEAM EFFORT TO SCRIPT DATA PROCESSING

##### data description here

In [ ]:
#Python ArcGIS import
import arcpy

In [ ]:
#Reproject TNC CAR dataset 2021
arcpy.management.Project("_15m_limite_imoveis_car_sicar_sem_sobrep", 
                         r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_SEM_SOBREP_TNC_2021_AEA", 
                         'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["Latitude_Of_Origin",-12.0],UNIT["Meter",1.0]]', 
                         None, 'GEOGCS["SIRGAS 2000",DATUM["Sistema_de_Referencia_Geocentrico_para_America_del_Sur_2000",SPHEROID["GRS 1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE",
                         None, "NO_VERTICAL")

#Reproject TNC settlements outside Conservation units dataset 2021
arcpy.management.Project("Assentamento_Brasil_sem_UC", 
                        r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\Assentamento_Brasil_sem_UC_AEA", 
                         'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["Latitude_Of_Origin",-12.0],UNIT["Meter",1.0]]',
                         None, 'GEOGCS["SIRGAS 2000",DATUM["D_Sistema_de_Referencia_Geocentrico_para_America_del_Sur_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", 
                         None, "NO_VERTICAL")

In [ ]:
#Settlements AOI selection 
assentamentos_AOI = arcpy.management.SelectLayerByLocation("Assentamento_Brasil_sem_UC_AEA;db00_escopo_municipio", "HAVE_THEIR_CENTER_IN", "db00_escopo_municipio", None, "NEW_SELECTION", "NOT_INVERT")

#Save settlements AOI 
arcpy.CopyFeatures_management(assentamentos_AOI, r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\Assentamento_Brasil_sem_UC_AEA_AOI")

#Clear selection db_escopo municipio
arcpy.SelectLayerByAttribute_management('db00_escopo_municipio', "CLEAR_SELECTION")

#CAR TNC AOI selection
CAR_TNC_AOI = arcpy.management.SelectLayerByLocation("CAR_SEM_SOBREP_TNC_2021_AEA;db00_escopo_municipio", "HAVE_THEIR_CENTER_IN", "db00_escopo_municipio", None, "NEW_SELECTION", "NOT_INVERT")

#Save CAR TNC AOI
arcpy.CopyFeatures_management(CAR_TNC_AOI, r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_SEM_SOBREP_TNC_2021_AEA_AOI")

In [ ]:
#Fiscal Module insertion and processing

#Join table -  fiscal module table into CAR shape
arcpy.management.JoinField("CAR_SEM_SOBREP_TNC_2021_AEA_AOI", "geocodigo", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_referencia\br_incra_modulos_fiscais_2013_por_municipio.xlsx\municipios_brasil$", "geocodigo", "MF_ha")

#Copy joined shapefile
arcpy.CopyFeatures_management('CAR_SEM_SOBREP_TNC_2021_AEA_AOI', r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF")

#Calculate field 'AREA' rural properties / MF_HA (geocodigo based)
arcpy.management.CalculateField("CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF", "MF_PROP", "!AREA_IMOVE! / !MF_ha!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#Create a rule to classify in MF_STATUS the small properties (< 4 MF)
arcpy.management.CalculateField("CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF", "MF_STATUS", "reclass(!MF_STATUS!, !MF_PROP!)", "PYTHON3", """def reclass(MF_STATUS, MF_PROP):
    if (MF_PROP < 4):
        mf_4 = 'Peq propriedade'
        return mf_4
    else:
        return 0""", "TEXT", "NO_ENFORCE_DOMAINS")

#Select only small properties
arcpy.management.SelectLayerByAttribute("CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF", "NEW_SELECTION", "MF_STATUS = 'Peq propriedade'", None)

#Save shapefile
arcpy.CopyFeatures_management('CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF', r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF_PP")

In [ ]:
#Union processing preparation

#Remove Multiple Overlapping polygons from Assentamentos dataset
arcpy.analysis.RemoveOverlapMultiple("Assentamento_Brasil_sem_UC_AEA_AOI", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\Assentamento_Brasil_sem_UC_AEA_AOI_RMOVLP.shp", "THIESSEN", "ALL")

#CAR and Settlements REPAIR GEOMETRIES
arcpy.management.RepairGeometry("CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF_PP", "DELETE_NULL", "ESRI")
arcpy.management.RepairGeometry("Assentamento_Brasil_sem_UC_AEA_AOI_RMOVLP", "DELETE_NULL", "ESRI")

#CAR and settlements union
arcpy.analysis.Union("CAR_SEM_SOBREP_TNC_2021_AEA_AOI_MF_PP #;Assentamento_Brasil_sem_UC_AEA_AOI_RMOVLP #", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_ASSENT_AOI_UNION.shp", "ALL", None, "GAPS")

In [ ]:
#Spatial Join to set municipalities in the dataset with "center in" criteria
arcpy.analysis.SpatialJoin("CAR_ASSENT_AOI_UNION", "db00_escopo_municipio", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_ASSENT_AOI_UNION_MUN.shp", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'FID_CAR_SE "FID_CAR_SE" true true false 10 Long 0 10,First,#,CAR_ASSENT_AOI_UNION,FID_CAR_SE,-1,-1;COD_IMOVEL "COD_IMOVEL" true true false 100 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,COD_IMOVEL,0,100;AREA_IMOVE "AREA_IMOVE" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,AREA_IMOVE,-1,-1;IND_STATUS "IND_STATUS" true true false 2 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,IND_STATUS,0,2;IND_TIPO_I "IND_TIPO_I" true true false 3 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,IND_TIPO_I,0,3;ir_ha "ir_ha" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,ir_ha,-1,-1;obs_sobrep "obs_sobrep" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,obs_sobrep,0,254;irrev_ha "irrev_ha" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,irrev_ha,-1,-1;porc_area "porc_area" true true false 10 Long 0 10,First,#,CAR_ASSENT_AOI_UNION,porc_area,-1,-1;nome_corre "nome_corre" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,nome_corre,0,254;geocodigo "geocodigo" true true false 80 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,geocodigo,0,80,db00_escopo_municipio,geocodigo,0,80;sigla "sigla" true true false 2 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,sigla,0,2,db00_escopo_municipio,sigla,0,2;A__D2007 "A__D2007" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2007,-1,-1;A__D2008 "A__D2008" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2008,-1,-1;A__D2009 "A__D2009" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2009,-1,-1;A__D2010 "A__D2010" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2010,-1,-1;A__D2011 "A__D2011" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2011,-1,-1;A__D2012 "A__D2012" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2012,-1,-1;A__D2013 "A__D2013" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2013,-1,-1;A__D2014 "A__D2014" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2014,-1,-1;A__D2015 "A__D2015" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2015,-1,-1;A__D2016 "A__D2016" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2016,-1,-1;A__D2017 "A__D2017" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2017,-1,-1;A__D2018 "A__D2018" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2018,-1,-1;A__D2019 "A__D2019" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2019,-1,-1;A__D2020 "A__D2020" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,A__D2020,-1,-1;regiao "regiao" true true false 50 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,regiao,0,50,db00_escopo_municipio,regiao,0,50;Desmat_ant "Desmat_ant" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,Desmat_ant,-1,-1;Desmat_pos "Desmat_pos" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,Desmat_pos,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,Shape_Leng,-1,-1,db00_escopo_municipio,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,Shape_Area,-1,-1,db00_escopo_municipio,Shape_Area,-1,-1;MF_ha "MF_ha" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,MF_ha,-1,-1;MF_PROP "MF_PROP" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,MF_PROP,-1,-1;MF_STATUS "MF_STATUS" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,MF_STATUS,0,254;FID_Assent "FID_Assent" true true false 10 Long 0 10,First,#,CAR_ASSENT_AOI_UNION,FID_Assent,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,CAR_ASSENT_AOI_UNION,ORIG_FID,-1,-1;cd_sipra "cd_sipra" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,cd_sipra,0,254;uf "uf" true true false 2 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,uf,0,2;nome_proje "nome_proje" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,nome_proje,0,254;municipio "municipio" true true false 254 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,municipio,0,254;area_hecta "area_hecta" true true false 40 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,area_hecta,0,40;capacidade "capacidade" true true false 9 Long 0 9,First,#,CAR_ASSENT_AOI_UNION,capacidade,-1,-1;num_famili "num_famili" true true false 9 Long 0 9,First,#,CAR_ASSENT_AOI_UNION,num_famili,-1,-1;fase "fase" true true false 9 Long 0 9,First,#,CAR_ASSENT_AOI_UNION,fase,-1,-1;data_de_cr "data_de_cr" true true false 10 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,data_de_cr,0,10;forma_obte "forma_obte" true true false 50 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,forma_obte,0,50;data_obten "data_obten" true true false 10 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,data_obten,0,10;area_calc_ "area_calc_" true true false 19 Double 4 18,First,#,CAR_ASSENT_AOI_UNION,area_calc_,-1,-1;sr "sr" true true false 50 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,sr,0,50;descricao_ "descricao_" true true false 50 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,descricao_,0,50;tipo "tipo" true true false 50 Text 0 0,First,#,CAR_ASSENT_AOI_UNION,tipo,0,50;area_ha "area_ha" true true false 19 Double 0 0,First,#,CAR_ASSENT_AOI_UNION,area_ha,-1,-1,db00_escopo_municipio,area_ha,-1,-1;Join_Count "Join_Count" true true false 10 Long 0 10,First,#,db00_escopo_municipio,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 10 Long 0 10,First,#,db00_escopo_municipio,TARGET_FID,-1,-1;JOIN_FID "JOIN_FID" true true false 10 Long 0 10,First,#,db00_escopo_municipio,JOIN_FID,-1,-1;Join_Cou_1 "Join_Cou_1" true true false 10 Long 0 10,First,#,db00_escopo_municipio,Join_Cou_1,-1,-1;nome "nome" true true false 150 Text 0 0,First,#,db00_escopo_municipio,nome,0,150;geocod_n "geocod_n" true true false 10 Long 0 10,First,#,db00_escopo_municipio,geocod_n,-1,-1;nome_1 "nome_1" true true false 80 Text 0 0,First,#,db00_escopo_municipio,nome_1,0,80;geocodig_1 "geocodig_1" true true false 80 Text 0 0,First,#,db00_escopo_municipio,geocodig_1,0,80;nome_calta "nome_calta" true true false 250 Text 0 0,First,#,db00_escopo_municipio,nome_calta,0,250;ibge_ha "ibge_ha" true true false 10 Long 0 10,First,#,db00_escopo_municipio,ibge_ha,-1,-1', "HAVE_THEIR_CENTER_IN", None, '')

In [ ]:
# Select not null data in municipalities
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN", "NEW_SELECTION", "nome <> ' '", None)

In [ ]:
# Save shapefile
#arcpy.CopyFeatures_management('CAR_ASSENT_AOI_UNION_MUN', r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_ASSENT_AOI_UNION_MUN_f")

#deleting undesirable fields from main shapefile
#arcpy.management.DeleteField("CAR_ASSENT_AOI_UNION_MUN_f", "Join_Count;TARGET_FID;FID_CAR_SE;nome_corre;geocodigo;sigla;A__D2007;A__D2008;A__D2009;A__D2010;A__D2011;A__D2012;A__D2013;A__D2014;A__D2015;A__D2016;A__D2017;A__D2019;A__D2020;regiao;Desmat_ant;Desmat_pos;Shape_Leng;Shape_Area;FID_Assent;ORIG_FID;area_ha;Join_Cou_1;TARGET_F_1;JOIN_FID;Join_Cou_2;nome_calta;ibge_ha", "DELETE_FIELDS")
#arcpy.management.DeleteField("CAR_ASSENT_AOI_UNION_MUN_f", "Join_Cou_1;TARGET_F_1;JOIN_FID;Join_Cou_2;nome_1;geocodig_1;nome_calta;ibge_ha", "DELETE_FIELDS")

In [ ]:
#Considering "assentamento" info on column COD_IMOVEL to classify settlements and CAR data classes
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_f", "NEW_SELECTION", "COD_IMOVEL = ''", None)
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_f", "COD_IMOVEL", '"Assentamento"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_f', "CLEAR_SELECTION")

In [ ]:
### DATABASE PROCESSING UP TO NOW, STARTING TO PROCESS TIF FILES (PRODES, PASTURE AREAS)

In [ ]:
# processing raster to reproject - Albers Equal Area
#arcpy.management.ProjectRaster("PDigital2000_2021_AMZ.tif", 
#                               r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\PDigital2000_2021_AMZ_AEA.tif", 
#                               'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["Latitude_Of_Origin",-12.0],UNIT["Meter",1.0]]', 
#                               "NEAREST", "30 30", None, None, 'GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")
            
# transforming PRODES 2021 to polygon reference
#arcpy.conversion.RasterToPolygon("PDigital2000_2021_AMZ_AEA.tif", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\PDigital2000_2021_AMZ_AEA.shp", "SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

# Pasture LAPIG MAPBIOMAS COLL 5 reproject
#arcpy.management.ProjectRaster("mapbiomas-brazil-collection-50-pasture-quality-limiteestadosdaamazonia-2018.tif", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\mapbiomas-brazil-collection-50-pasture-quality-limiteestadosdaamazonia-2018_AEA.tif", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["Latitude_Of_Origin",-12.0],UNIT["Meter",1.0]]', "NEAREST", "30 30", None, None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")

In [ ]:
# intersect PRODES 2007~2021 dataset with main shapefile (CAR + Settlements)
arcpy.analysis.Intersect("PDigital2000_2021_AMZ_AEA #;CAR_ASSENT_AOI_UNION_MUN_f #", 
                         r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_ASSENT_AOI_UNION_MUN_PD2021.shp", 
                         "ALL", None, "INPUT")

In [ ]:
# Tabulate area Pasture Classes and CAR_ASSENT with PRODES 2007~2021 gridcode
arcpy.sa.TabulateArea("CAR_ASSENT_AOI_UNION_MUN_PD2021", "FID", 
                      "mapbiomas-brazil-collection-50-pasture-quality-limiteestadosdaamazonia-2018_AEA.tif", "Value", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\Tabulate_Pasto1",
                      r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\mapbiomas-brazil-collection-50-pasture-quality-limiteestadosdaamazonia-2018_AEA.tif", 
                      "CLASSES_AS_FIELDS")

# Join tabulate with main shapefile
arcpy.management.JoinField("CAR_ASSENT_AOI_UNION_MUN_PD2021", "FID", "tabulate_pasto1", "FID", None)

In [ ]:
# Save shapefile with PRODES 2007~2021 and Pasture classes
arcpy.CopyFeatures_management('CAR_ASSENT_AOI_UNION_MUN_PD2021', 
                              r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE")

In [ ]:
#calculate area_ha field considering intersects between PRODES and CAR/Settlements database
arcpy.management.CalculateGeometryAttributes("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "ar_ha_all AREA_GEODESIC", '', "HECTARES", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

In [ ]:
# CLOSING THE PROCESSING STEP TO INCLUDE PRODES AND PASTURE

In [ ]:
#PIVOT TABLE PRODES
#calculate field
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "ID_PV", "!FID!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
#pivot table processing
arcpy.management.PivotTable("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "ID_PV;gridcode;ar_ha_all", "gridcode", "ar_ha_all", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_projeto\Projeto.gdb\CAR_ASSENT_AOI_UNION_MUN_PD24")
#join to shapefile
arcpy.management.JoinField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "ID_PV", "CAR_ASSENT_AOI_UNION_MUN_PD24", "ID_PV", None)

In [ ]:
# organizing fields pasture
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "SEM_PASTO", "!VALUE_0! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "PASTO_NDEG", "!VALUE_1! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "PASTO_LEVE", "!VALUE_2! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "PASTO_MOD", "!VALUE_3! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "PASTO_SEV", "!VALUE_4! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#Delete old pasture fields 
# area_ha is the old area_ha calculated before intersect with PRODES (not valid)
arcpy.management.DeleteField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE",
                             "VALUE_0;VALUE_1;VALUE_2;VALUE_3;VALUE_4;ID_PV;ID_PV_1;gridcode_1;area_ha", 
                             "DELETE_FIELDS")

In [ ]:
#PRODES fields organization
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2007", "!gridcode6!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2008", "!gridcode7!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2009", "!gridcode8!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2010", "sum([!gridcode9!, !gridcode18!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2011", "sum([!gridcode10!, !gridcode19!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2012", "sum([!gridcode11!, !gridcode20!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2013", "sum([!gridcode12!, !gridcode21!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2014", "sum([!gridcode13!, !gridcode22!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2015", "sum([!gridcode14!, !gridcode23!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2016", "sum([!gridcode15!, !gridcode24!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2017", "sum([!gridcode16!, !gridcode25!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2018", "sum([!gridcode17!, !gridcode26!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2019", "sum([!gridcode27!, !gridcode28!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2020", "sum([!gridcode29!, !gridcode31!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D2021", "!gridcode33!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

# delete old PRODES fields
arcpy.management.DeleteField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "gridcode1;gridcode10;gridcode11;gridcode12;gridcode13;gridcode14;gridcode15;gridcode16;gridcode17;gridcode18;gridcode19;gridcode2;gridcode20;gridcode21;gridcode22;gridcode23;gridcode24;gridcode25;gridcode26;gridcode27;gridcode28;gridcode29;gridcode3;gridcode31;gridcode32;gridcode33;gridcode4;gridcode6;gridcode7;gridcode8;gridcode9", "DELETE_FIELDS")

In [ ]:
# Imazon_v2019 vicinals and secondary roads 
#LOGISTICS AND DISTANCE FROM ROADS ANALYSIS

#REPROJECT
#arcpy.management.Project("estradas_bioma_2012_geo", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\01_base_cartografica\imazon_v2019\estradas_bioma_2012_geo_AEA.shp", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SAD_1969_To_WGS_1984_14", 'GEOGCS["GCS_South_American_1969",DATUM["D_South_American_1969",SPHEROID["GRS_1967_Truncated",6378160.0,298.25]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

# NEAR PROCESSING - FIELD NEAR ROAD
arcpy.analysis.Near("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "estradas_bioma_2012_geo_AEA", None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_ROAD")

In [ ]:
# Deforestment analysis up to 2008 Column area
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D_ANT2008", "sum([!D2007!, !D2008!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
#classx(!D_ANT2008t! , !D_ANT2008_!)

#def classx(D_ANT2008t, D_ANT2008_):
#    if (D_ANT2008_ == 0):
#        desmat = "sem desmatamento anterior 2008"
#        return desmat
#    elif (D_ANT2008_ > 0):
#        desmat = "desmatou antes/até 2008"
#        return desmat
 
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D_ANT2008t", "classx(!D_ANT2008t! , !D_ANT2008!)", "PYTHON3", """def classx(D_ANT2008t, D_ANT2008):
    if (D_ANT2008 == 0):
        desmat = "sem desmatamento anterior 2008"
        return desmat
    elif (D_ANT2008 > 0):
        desmat = "desmatou antes/até 2008"
        return desmat
        """, "TEXT", "NO_ENFORCE_DOMAINS")

In [ ]:
# Deforestment analysis post 2008 column area
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D_POS2008", "sum([!D2009! , !D2010! , !D2011!, !D2012! ,!D2013! , !D2014!, !D2015!, !D2016!, !D2017!, !D2018!, !D2019!, !D2020!, !D2021!])", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
#class_pos2008(!D_POS2008t! , !D_POS2008!)

#def class_pos2008(D_POS2008t, D_POS2008):
#    if (D_POS2008 == 0):
#        desmat = "sem desmatamento pós 2008"
#        return desmat
#    elif (D_POS2008 > 0):
#        desmat = "desmatou pós 2008"
#        return desmat

arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "D_POS2008t", "class_pos2008(!D_POS2008t! , !D_POS2008!)", "PYTHON3", """def class_pos2008(D_POS2008t, D_POS2008):
    if (D_POS2008 == 0):
        desmat = "sem desmatamento após 2008"
        return desmat
    elif (D_POS2008 > 0):
        desmat = "desmatou após 2008"
        return desmat""", "TEXT", "NO_ENFORCE_DOMAINS")

In [ ]:
# Main soil types with naturally high suitability for producing cocoa 
#(Theobroma cacao) at low input level in Para´ state
''
arcpy.management.SelectLayerByLocation("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "INTERSECT", "Solos_Schroth2016_AEA_AOI", None, "NEW_SELECTION", "NOT_INVERT")

arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "SOIL", "'SUITABLE TO COCOA'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', "CLEAR_SELECTION")

In [ ]:
# NEAR UCS PI US CONSIDERING DATA SOURCE
arcpy.analysis.Near("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "br_uc_mma_UCs_PI_US_AEA", None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_UCS")

In [ ]:
# APA WITHIN SELECTION CONSIDERING DATA SOURCE
arcpy.management.SelectLayerByLocation("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "WITHIN", "br_uc_mma_APA_AEA", None, "NEW_SELECTION", "NOT_INVERT")

# Create column APA and fill with "Sim", considering previous selection
arcpy.management.CalculateField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "APA", '"Sim"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

#clear selection
arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', "CLEAR_SELECTION")

In [ ]:
# NEAR INDIGENOUS TERRITORIES (2hrs processing)
arcpy.analysis.Near("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "br_ti_funai_AEA;br_ti_estudo_funai_AEA", None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_TI")

In [ ]:
# Join area plantada cacau (ha_p), área cultivada (ha_c) e produtividade (_ton)
arcpy.management.JoinField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "geocod_n", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_referencia\Dados_Cacau_SEDAP_Para\Dados_Cacau_SEDAP_cleaned.xlsx\T_Cacau__em_amêndoa_$_", "cod", None)

In [1]:
## BC 1:250k IBGE v2021 lml_cidades_pontos - Análise de sedes municipais 

#REPROJECT
#arcpy.management.Project("lml_cidade_p", r"C:\Users\guilherme.fronza\Downloads\IBGE_BC250_v2021\lml_cidade_pontos_AEA", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SIRGAS_2000_To_WGS_1984_1", 'GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

#FILTER CONSIDERING AOI MUNICIPALITIES
#arcpy.management.SelectLayerByLocation("lml_cidade_pontos_AEA;db00_escopo_municipio", "COMPLETELY_WITHIN", "db00_escopo_municipio", None, "NEW_SELECTION", "NOT_INVERT")

#Save shapefile
#arcpy.CopyFeatures_management('lml_cidade_pontos_AEA', r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\lml_cidade_pontos_AEA_AOI")

#Clear selection
#arcpy.SelectLayerByAttribute_management('lml_cidade_pontos_AEA', "CLEAR_SELECTION")

<Result 'lml_cidade_pontos_AEA'>

In [4]:
# NEAR PROCESSING - FIELD NEAR SED
arcpy.analysis.Near("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "lml_cidade_pontos_AEA_AOI", None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_FID FID_SED;NEAR_DIST NEAR_SED")

<Result 'CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE'>

In [5]:
#Mining extraction near analysis
# considering TNC TI's database, SFX Tucumã 2018 database, IBGE 250k database (points and polygons)

#reprojections
arcpy.management.Project("eco_ext_mineral_a", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\eco_ext_mineral_a_AEA", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SIRGAS_2000_To_WGS_1984_1", 'GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("mining_2018_mun", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\mining_2018_mun_AEA", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SIRGAS_2000_To_WGS_1984_1", 'PROJCS["SIRGAS_2000_UTM_Zone_22S",GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",-51.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("mineracai_ti_pa", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\mineracai_ti_pa_AEA", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SAD_1969_To_WGS_1984_15", 'PROJCS["South_America_Albers_Equal_Area_Conic",GEOGCS["GCS_South_American_1969",DATUM["D_South_American_1969",SPHEROID["GRS_1967_Truncated",6378160.0,298.25]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-60.0],PARAMETER["Standard_Parallel_1",-5.0],PARAMETER["Standard_Parallel_2",-42.0],PARAMETER["Latitude_Of_Origin",-32.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project("eco_ext_mineral_p", r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\eco_ext_mineral_p_AEA", 'PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["central_meridian",-54.0],PARAMETER["Standard_Parallel_1",-2.0],PARAMETER["Standard_Parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]', "SIRGAS_2000_To_WGS_1984_1", 'GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'C:\\Users\\guilherme.fronza\\OneDrive\\cacauFlorestaAcelerador\\amazon_acelerador\\04a_dados_espaciais\\02_dados_automacao\\eco_ext_mineral_p_AEA'>

In [1]:
# NEAR MINING ANALYISIS (NO SUBSET TO AOI)
arcpy.analysis.Near("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "eco_ext_mineral_p_AEA;mining_mapbiomas_coverage2020_AEA_AOI;mining_2018_mun_AEA;mineracai_ti_pa_AEA;eco_ext_mineral_a_AEA", None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_MIN")

<Result 'CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE'>

In [ ]:
#### filtering database to build data

In [11]:
###Amazon Selection 10 years considering no deforestment areas 
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "NEW_SELECTION", "D2021 = 0 And D2020 = 0 And D2019 = 0 And D2018 = 0 And D2017 = 0 And D2016 = 0 And D2015 = 0 And D2014 = 0 And D2013 = 0 And D2012 = 0", None)

# Light or moderate pasture quality filter
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "SUBSET_SELECTION", "PASTO_LEVE > 0 Or PASTO_MOD > 0", None)

#Create new shapefile 
arcpy.CopyFeatures_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', 
                              r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao/DB_10YEARS_PLEVMOD")

#Clear selection 
arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', "CLEAR_SELECTION")


<Result 'CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE'>

In [15]:
###Amazon Selection 5 years considering no deforestment areas 
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "NEW_SELECTION", "D2021 = 0 And D2020 = 0 And D2019 = 0 And D2018 = 0 And D2017 = 0", None)

# Light or moderate pasture quality filter
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "SUBSET_SELECTION", "PASTO_LEVE > 0 Or PASTO_MOD > 0", None)

#Create new shapefile 
arcpy.CopyFeatures_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', 
                              r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao/DB_5YEARS_PLEVMOD")

#Clear selection 
arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', "CLEAR_SELECTION")

<Result 'CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE'>

In [2]:
###Amazon Selection 3 years considering no deforestment areas 
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "NEW_SELECTION", "D2021 = 0 And D2020 = 0 And D2019 = 0 And D2018 = 0", None)

# Light or moderate pasture quality filter
arcpy.management.SelectLayerByAttribute("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "SUBSET_SELECTION", "PASTO_LEVE > 0 Or PASTO_MOD > 0", None)

#Create new shapefile 
arcpy.CopyFeatures_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', 
                              r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02a_dados_automacao\4YEARS_PLEVMOD_CF")

#Clear selection 
arcpy.SelectLayerByAttribute_management('CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE', "CLEAR_SELECTION")

<Result 'CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE'>

In [1]:
### processing to Selecao Natural study - functional connectivity

out_raster = arcpy.sa.Reclassify("brasil_coverage_2020_AEA_PARA.tif", "Value", "3 1;4 1;5 1;9 0;11 1;12 1;15 0;20 0;21 0;23 0;24 0;25 0;30 0;32 1;33 0;39 0;41 0;48 0", "DATA"); out_raster.save(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_automacao\brasil_coverage_2020_AEA_PARA_HABMAT.tif")

In [ ]:
#PREVIOUS DEVELOPMENT (Version DEZ/21)

In [ ]:
# brasil mapbiomas coverage 
#arcpy.management.ProjectRaster("brasil_coverage_2020.tif", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\Mapbiomas_Brazil_2020_coverage_AEA.tif", 'PROJCS["IBGE_Albers_Equal_Area_Conic_Amz",GEOGCS["GCS_SIRGAS_2000",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-59.0],PARAMETER["Standard_Parallel_1",1.0],PARAMETER["Standard_Parallel_2",-14.0],PARAMETER["Latitude_Of_Origin",5.0],UNIT["Meter",1.0]]', "NEAREST", "30 30", "SIRGAS_2000_To_WGS_1984_1", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")

# Clip raster mapbiomas coverage 2020 to AOI
#arcpy.management.Clip("Mapbiomas_Brazil_2020_coverage_AEA.tif", "254373.171972426 -1656805.00945003 1436752.68317566 -610634.994524859", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\Mapbiomas_Brazil_2020_coverage_AEA_AOI.tif", "db00_escopo_municipio", "0", "NONE", "NO_MAINTAIN_EXTENT")

# extract forest class (value 3)
#arcpy.ddd.Reclassify("Mapbiomas_Brazil_2020_coverage_AEA_AOI.tif", "Value", "3 1;4 0;5 0;9 0;11 0;12 0;15 0;20 0;21 0;23 0;24 0;25 0;30 0;32 0;33 0;39 0;41 0;48 0", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\Mapbiomas_Brazil_2020_coverage_AEA_AOI_CL3.tif", "DATA")

# tabulate area 
#arcpy.sa.TabulateArea("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "FID", "Mapbiomas_Brazil_2020_coverage_AEA_AOI_CL3.tif", "Value", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\tabulate_flores_mask", r"C:\Users\guilherme.fronza\OneDrive\Amazon_Select\Automation\Mapbiomas_Brazil_2020_coverage_AEA_AOI_CL3.tif", "CLASSES_AS_FIELDS")

# rename mask flo value 0 field and divide by 10000 to get area in ha
#arcpy.management.CalculateField("tabulate_flores_mask", "sem_desm", "!VALUE_0! / 10000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

# tabulate area in shapefile 
#arcpy.management.JoinField("CAR_ASSENT_AOI_UNION_MUN_PD2021_PASTURE", "FID", "tabulate_flores_mask", "FID", "SEM_DESM")